## <font color = yellow> A single Sale will combine the first row from the Buys & Sells dfs

In [1]:
library(tibble)

In [2]:
create_df <- function() {
    
    df <- read.csv("fifo_new.csv", header = TRUE, sep = ",")
    df$sale <- as.integer(df$sale)
   #  df$date <- as.Date(df$date, origin = "1899-12-30")
   #  df$true_false <- FALSE  
    return(df)  
    }
i <- 1
# fct call
# df <- create_df() 

In [3]:
# call fct and create the record keeping df
df <- create_df()
# create empty df
df_records <- data.frame()

# df_buy_rows <- data.frame()

In [4]:
# create dfs from filtered df
buys <- df[df$type == 'Buy', ]  # Filter buys
sells <- df[df$type == 'Sell', ]  # Filter sells
head(buys, 3)
head(sells,3)

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,cf_pps,cf_basis,buy_quan,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sale
,<int>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>
1,1,44926,Buy,AMC,1050,4.070,4273.5,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,44946,Buy,AMC,1000,5.625,5625.0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,5,44949,Buy,AMC,1000,5.788,5788.0,0,0,0,0,0,0,0,0,0,0,0,0,0


,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,cf_pps,cf_basis,buy_quan,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sale
,<int>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>
2,2,44953,Sell,AMC,1000,5.51240,5512.40,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,44960,Sell,AMC,1000,6.68255,6682.55,0,0,0,0,0,0,0,0,0,0,0,0,0
6,6,44960,Sell,AMC,1000,6.25500,6255.00,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
# A combo fct that creates the dfs used (combo of above )

create_dfs <- function() {
    #fct call
    df <- create_df()   # the first row in this Notebook
    df_records <- data.frame()
    buys <- df[df$type == 'Buy', ]  # Filter buys
    sells <- df[df$type == 'Sell', ]  # Filter sells
    
    return(list(df = df, df_records = df_records, buys = buys, sells = sells))
  }
    

In [6]:
# Call the combo function, will return a list of dfs
dfs <- create_dfs()

# Access the dataframes from returned list
df <- dfs$df
df_records <- dfs$df_records
buys <- dfs$buys
sells <- dfs$sells


    

In [7]:
# create new dfs that will become the one line dfs (first row ) from buys and sells dfs
s <- data.frame()
b <- data.frame()

# create the Buy & Sell dfs

row_b <- function(b) {
        b <- b
        b <- buys[1,]
        return(b)
    }

row_s <- function(s) {
        s <- s
        s <- sells[1,]
        return(s)
    }

    

# call fcts


b <- row_b(b)   
s <- row_s(s)    

nrow(b)
nrow(s)

print(b)
print(s)


[1] 1

[1] 1

  index  date type  co quan  pps  basis sale_quan tally cf_quan cf_pps cf_basis
1     1 44926  Buy AMC 1050 4.07 4273.5         0     0       0      0        0
  buy_quan buy_pps buy_basis sell_quan sell_pps sell_basis profit_loss sale
1        0       0         0         0        0          0           0    0
  index  date type  co quan    pps  basis sale_quan tally cf_quan cf_pps
2     2 44953 Sell AMC 1000 5.5124 5512.4         0     0       0      0
  cf_basis buy_quan buy_pps buy_basis sell_quan sell_pps sell_basis profit_loss
2        0        0       0         0         0        0          0           0
  sale
2    0


## <font color = yellow> These values get duplicated b/c no change of i value -  (no loop yet)

In [8]:
# b <- when_cf(b)
b

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,cf_pps,cf_basis,buy_quan,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sale
,<int>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>
1,1,44926,Buy,AMC,1050,4.07,4273.5,0,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
# Binds the base (unchanged) Buy row to the df_records
# The df_record will bind and increment each new (empty) b df that comes into this fct

#  tested using incrementing values of i which increment b to df_records





df_create_records <- function (df_records, b, i) {
    
    df_records <- df_records
    b <- b   
    i <- i
   
    df_records <- rbind(b, df_records)     

    i <<- i + 1
    
    return(df_records)
    
    }
i
nrow(df_records)



[1] 1

[1] 0

### testing in next 2 rows

In [10]:
# i <- 1

In [11]:
#  b <- buys[i+1 ,]
# df_records <- df_create_records(df_records, b, i)
# i
# df_records


In [12]:
i

[1] 1

In [13]:
# These values might be default when there is a carryforward ( Buy > Sell )

when_cf <- function (b, s, df_records) {
    
    b <- b
    s <- s
    df_records <- df_records
    
    
    if(nrow(df_records) == 0) {
        cat("The df_records is an empty df", "\n")
        } else if(nrow(df_records) > 0 ) {
        cat("The df_records has one or more rows of data ", "\n")
        
        }
   
    sale <- s$quan
    bs_ratio <- b$quan / s$quan
    cf_quan <-  b$quan - s$quan
    
    # assign the cf values - if they exist
    if(b$quan > s$quan) {
        b$cf_quan <- (b$quan - s$quan)        
        b$cf_pps <- b$pps
        b$cf_basis <- cf_quan * b$pps
    }
    b$sale_quan <- b$quan
    b$buy_quan <- s$quan 
    b$buy_pps <-  b$pps
    b$buy_basis <-  b$buy_quan * b$pps

    b$sell_quan <- s$quan
    b$sell_pps <- s$pps
    b$sell_basis <- s$quan * s$pps

    b$profit_loss <- b$sell_basis - b$buy_basis
    b$sale <- sale
    
    s$type <- "SOLD"
    
    return(b)
    
    }
    

In [14]:
df_records

<0 x 0 matrix>

In [15]:
b
s

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,cf_pps,cf_basis,buy_quan,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sale
,<int>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>
1,1,44926,Buy,AMC,1050,4.07,4273.5,0,0,0,0,0,0,0,0,0,0,0,0,0


,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,cf_pps,cf_basis,buy_quan,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sale
,<int>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>
2,2,44953,Sell,AMC,1000,5.5124,5512.4,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
df_records <- when_cf(b, s, df_records)
df_records

The df_records is an empty df 


,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,cf_pps,cf_basis,buy_quan,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sale
,<int>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>
1,1,44926,Buy,AMC,1050,4.07,4273.5,1050,0,50,4.07,203.5,1000,4.07,4070,1000,5.5124,5512.4,1442.4,1000


In [17]:
b

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,cf_pps,cf_basis,buy_quan,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sale
,<int>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>
1,1,44926,Buy,AMC,1050,4.07,4273.5,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
b

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,cf_pps,cf_basis,buy_quan,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sale
,<int>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>
1,1,44926,Buy,AMC,1050,4.07,4273.5,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
# Row 1 of Buys was deleted.  If there is a c/f in df_records on row 1, then 
# copy/bind that data to buys, row 1 .... 50 shares will become Buy, row 1


test_for_cf <- function(b, buys, df_records) {

    b <- b
    buys <- buys
    df_records <- df_records
    
    if(df_records$cf_quan > 0 ) {
        
        temp <- b
        
        temp$quan <- df_records$cf_quan
        temp$basis <- df_records$cf_basis
        temp$type <- "CF"

        i <- 1      
        
        # insert row into buys
        buys <- add_row(buys, temp, .before = i)        
        
        }
   } # fct

In [20]:
head(buys, 3)

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,cf_pps,cf_basis,buy_quan,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sale
,<int>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>
1,1,44926,Buy,AMC,1050,4.070,4273.5,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,44946,Buy,AMC,1000,5.625,5625.0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,5,44949,Buy,AMC,1000,5.788,5788.0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
#  These functions will delete the first row from the Buys and Sells dfs 
#  concluding the first Sale event

i <- 1


# fct
buy_snip <- function (b, buys, df_records) {
    
    b <- b
    buys <- buys    
    df_records <- df_records
    
    # remove the first row of buys ... is b 
    buys <- buys[-1, ]   
    
    
    if(df_records$cf_quan > 0 ) {
        
        #fct call
        buys <- test_for_cf(b, buys, df_records)
        return(buys)
        }    
    return(buys)
   }


# fct
sell_snip <- function(sells) {
    sells <- sells    
    sells <- sells[-1, ]    
    return(sells)
    }


# fct call


# nrow(buys)
# nrow(sells)

# buys <- buy_snip(buys)
# sells <- sell_snip(sells)
# nrow(buys)
# nrow(sells)
# head(buys, 3)

In [22]:
df_records

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,cf_pps,cf_basis,buy_quan,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sale
,<int>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>
1,1,44926,Buy,AMC,1050,4.07,4273.5,1050,0,50,4.07,203.5,1000,4.07,4070,1000,5.5124,5512.4,1442.4,1000


In [23]:
# These are the fcts (in order) to process a Sals, from start to end 

nrow(buys)
nrow(sells)

buys <- buy_snip(b,buys, df_records)
sells <- sell_snip(sells)
nrow(buys)
nrow(sells)
head(buys, 3)

[1] 23

[1] 18

[1] 23

[1] 17

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,cf_pps,cf_basis,buy_quan,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sale
,<int>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>
1,1,44926,CF,AMC,50,4.070,203.5,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,44946,Buy,AMC,1000,5.625,5625.0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,5,44949,Buy,AMC,1000,5.788,5788.0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
# fct call: to create vars from the first row of Buy & Sell
b <- row_b(b)   
s <- row_s(s)  
b
s

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,cf_pps,cf_basis,buy_quan,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sale
,<int>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>
1,1,44926,CF,AMC,50,4.07,203.5,0,0,0,0,0,0,0,0,0,0,0,0,0


,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,cf_pps,cf_basis,buy_quan,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sale
,<int>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>
4,4,44960,Sell,AMC,1000,6.68255,6682.55,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
df_records
head(buys, 3)

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,cf_pps,cf_basis,buy_quan,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sale
,<int>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>
1,1,44926,Buy,AMC,1050,4.07,4273.5,1050,0,50,4.07,203.5,1000,4.07,4070,1000,5.5124,5512.4,1442.4,1000


,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,cf_pps,cf_basis,buy_quan,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,sale
,<int>,<int>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<int>
1,1,44926,CF,AMC,50,4.070,203.5,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,44946,Buy,AMC,1000,5.625,5625.0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,5,44949,Buy,AMC,1000,5.788,5788.0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
 stopps

ERROR: Error in eval(expr, envir, enclos): object 'stopps' not found


## <font color = yellow> Fresh Start: sequence of functions to process one FiFO Sale (one Buy/ one Sell) 
    
        dfs <- create_dfs() 
        row_b <- function(b) 
        row_s <- function(s) 
        when_cf  when_cf(b, s, df_buy_rows)
        df_records <- function (buys, b) 
        buy_snip <- function (buys) 
        sell_snip <- function(sells) 


In [ ]:
# Call the combo function named, create_dfs(), which returns a list of 4 new dfs

dfs <- create_dfs()
is.list(dfs)

# asign the dataframes from returned list
df <- dfs$df
df_ records <- 
df_buy_rows <- dfs$df_buy_rows
buys <- dfs$buys
sells <- dfs$sells

nrow(df)
nrow(df_buy_rows) # this df s/b empty (now)
nrow(buys)
nrow(sells)





In [ ]:
# fct call: populates the fields of B with the Sale data when there is a C/F amound
nrow(buys)
nrow(sells)

b <- when_cf(b, s, df_buy_rows) 

b    

In [ ]:
# The Buys df  - no changes yet
head(buys,3)

In [ ]:
# fct call:  First time the fct call creates new df called: records, from the df b,
#  Subsequent fct calls should append to the df_records for each call to this fct

buys <- df_records(buys, b, i) 

In [ ]:
# No changes yet
nrow(buys)
nrow(sells)

In [ ]:
# fct calls: to remove the first row from dfs

buys <- buy_snip(buys)
sells <- sell_snip(sells)

nrow(buys)
nrow(sells)

In [ ]:
# The Sale transaction was preserved in this df

head(df_buy_rows)

In [ ]:
stopps # end of loop 1

## <font color = yellow> Works as expected w/ one Buy - one Sell 
    
#### <font color = yellow> See what happens using the next Buy / Sell - loop 2 (involves the c/f from Sale 1 )
    
    dfs <- create_dfs() 
    row_b <- function(b) 
    row_s <- function(s) 
    
    b <- row_b(b)   
    s <- row_s(s)    
         
     when_cf(b, s, df_buy_rows)
    df_records <- function (buys, b) 
    buy_snip <- function (buys) 
    sell_snip <- function(sells) 
                 

    
    

In [ ]:
head(buys, 3)
head(sells, 3)

In [ ]:
# Run the fcts (w/out create_dfs)


b <- buys[1, ]
s <- sells[1, ]    

In [ ]:
print(b)
print(s)

In [ ]:
b <- when_cf(b, df_buy_rows)     
buys <- df_records(buys, b) 
buys <- buy_snip(buys)
sells <- sell_snip(sells)

In [ ]:
df_buy_rows